In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
import loadOSCdata
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = loadOSCdata.downloadData(OSCid=367385, X = True, Y = True, Z = True, output = 'csv', outputFile = 'data.csv')
data.head()

,timestamp,point_id,v_value,geometry_raw,image_url,image_lab,trip_id
792,2017-05-26 17:15:57.918,0.0,0.761113,POINT (-73.93671009000001 40.704303425),http://storage4.openstreetcam.org/files/photo/...,NaN,367385
3462,2017-05-26 17:16:13.945,1.0,8.750670,POINT (-73.936764505 40.70428552),http://storage4.openstreetcam.org/files/photo/...,NaN,367385
5727,2017-05-26 17:16:30.904,2.0,25.468589,POINT (-73.93671463999999 40.704234995),http://storage4.openstreetcam.org/files/photo/...,NaN,367385
6128,2017-05-26 17:16:34.900,3.0,27.397556,POINT (-73.93687019000001 40.704219665),http://storage4.openstreetcam.org/files/photo/...,NaN,367385
6434,2017-05-26 17:16:37.900,4.0,36.878148,POINT (-73.937003105 40.70423295000001),http://storage4.openstreetcam.org/files/photo/...,NaN,367385
